In [2]:
#installing the required libraries

from PyPDF2 import PdfReader 

In [60]:
reader = PdfReader('/home/roy_aakash/Downloads/mirror2_local/sample pdfs/Guideline_Investible_Weight_Factors.pdf')
text = ''
for page in reader.pages:
    text += page.extract_text()

In [61]:
text

' \nInvestible Weight Factors (IWFs)  \nIWF as the term suggests is a unit of floating stock expressed in terms of a number available for \ntrading and which is not held by the entities having strategic interest in a company. Higher IWF \nsuggest greater number of shares held by the investors as reported under public category within \na shareholding pattern reported by each company.  \n \nThe IWF  for each company in the index are determined based on the public shareholding of the \ncompanies as dis closed in the shareholding pattern submitted to the stock exchanges on \nquarterly basis. The following categories are excluded from the free float factor computation  \nwhere identifiable separately : \n \n1. Shareholding reported under promoter category  \n2. Depository Re ceipts (DRs) held by promoters and promoter groups  \n3. Associate/group companies/ cross -holdings  \n4. Family members of promoter  \n5. Trusts managed by promoter/ promoter group companies  \n6. Employee Benefit Trus

In [1]:
def slice_text(text, range_start, range_end):
    # Split the text into sentences
    sentences = text.split(". ")

    # Check if range indices are valid
    if range_start < 0 or range_end > len(sentences):
        return "Invalid range!"

    # Slice the sentences according to the given range
    sliced_text = ". ".join(sentences[range_start : range_end])

    # Append a dot at end if there was a dot at the end of last sentence
    if sentences[range_end - 1][-1] == ".":
        sliced_text += "."

    return sliced_text

In [62]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 256,
    chunk_overlap  = 20
)

chunks = text_splitter.create_documents([text])

In [63]:
n_chunks = []
for chunk in chunks:
    n_chunks.append(list(chunk)[0][1])

In [64]:
n_chunks

['Investible Weight Factors (IWFs)  \nIWF as the term suggests is a unit of floating stock expressed in terms of a number available for \ntrading and which is not held by the entities having strategic interest in a company. Higher IWF',
 'suggest greater number of shares held by the investors as reported under public category within \na shareholding pattern reported by each company.  \n \nThe IWF  for each company in the index are determined based on the public shareholding of the',
 'companies as dis closed in the shareholding pattern submitted to the stock exchanges on \nquarterly basis. The following categories are excluded from the free float factor computation  \nwhere identifiable separately :',
 '1. Shareholding reported under promoter category  \n2. Depository Re ceipts (DRs) held by promoters and promoter groups  \n3. Associate/group companies/ cross -holdings  \n4. Family members of promoter',
 '5. Trusts managed by promoter/ promoter group companies  \n6. Employee Benefit Tr

In [8]:
from sentence_transformers import SentenceTransformer

/home/roy_aakash/Downloads/mirror2_local/mirror_venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [11]:
import chromadb

In [12]:
client = chromadb.PersistentClient(path="/home/roy_aakash/Downloads/mirror2_local/mirror2/docs_vectorDB")

In [65]:
chunk_name = 'nse'

In [66]:
sample1_collection = client.create_collection(chunk_name)
sample1_collection.add(documents = n_chunks,
    ids = list(map(lambda tup: f"id{tup[0]}", enumerate(chunks))))

In [67]:
client.list_collections()

[Collection(name=nse_test_p),
 Collection(name=nse_test_c),
 Collection(name=nse)]

In [59]:
#client.delete_collection('sample1')

In [68]:
query = 'What is the purpose of Investible Weight Factors (IWFs) in relation to floating stock?'

In [69]:
result = sample1_collection.query(query_texts=[query], n_results=2, include=["documents", 'distances',]) #the way we query in chromadb
top_chunks = result['documents'][0]

In [70]:
for c in top_chunks:
    print(c)

Investible Weight Factors (IWFs)  
IWF as the term suggests is a unit of floating stock expressed in terms of a number available for 
trading and which is not held by the entities having strategic interest in a company. Higher IWF
suggest greater number of shares held by the investors as reported under public category within 
a shareholding pattern reported by each company.  
 
The IWF  for each company in the index are determined based on the public shareholding of the


In [71]:
import local_llm
from chunking import qa_processing

In [73]:
answer = local_llm.get_answer(qa_processing.make_categorize_conversation_prompt(query, top_chunks))

In [75]:
print(answer)

 Investible Weight Factors (IWFs) are a way for investors to gauge the value of floating shares in a company. They serve as a benchmark for comparing the worth of different stocks within an index, helping investors understand the relative performance and risk levels of companies within that group.

The primary purpose of IWFs is to provide investors with an insight into the true value of their investments in a company's floating shares, regardless of whether they hold any actual shares themselves. This information can be very useful for investors seeking to make informed decisions about their portfolios and understand how their funds are being managed.

However, it is also important to note that the exact number of floating shares available for trading depends on various factors including the company's capital structure, public shareholdings and the jurisdiction in which the company operates. This information should be considered when interpreting IWFs, as they are not a reliable measu